In [1]:
import torch

from torch import nn
from torch.utils import data
from torch.nn import functional as F

from torchvision import transforms
from random import randint

import sys
import os
from os import path

import pandas as pd

from functools import partial, reduce
from tqdm import tqdm

import datetime

## Local Imports ##
if '../' not in sys.path:
    sys.path.insert(0, '../')
from models import helpers as model_helpers, model_definitions as custom_models
from datasets import helpers as dataset_helpers, datasets as custom_datasets

from train_single_script import create_arg_str

from VOC import DT_DEST_RGB_RANDOM, DT_DEST_RGB_SINGLE_CLASS 

In [2]:
# Channel, Width, Height
C, W, H = (3, 224, 224)

TRAIN_SINGLE_PATH = './train_single_script.py'

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DT_ROOT = 'data'
POLYGON_COUNT_DIR = path.join(DT_ROOT, 'polygon_data_counts')
POLYGON_PERCENTAGE_DIR = path.join(DT_ROOT, 'polygon_data_percentage')

ELLIPSE_COUNT_DIR = path.join(DT_ROOT, 'ellipse_data_counts')
ELLIPSE_PERCENTAGE_DIR = path.join(DT_ROOT, 'ellipse_data_percentage')

VOC_SEGS_COUNTS_DIR = path.join('/home', 'victor', 'datasets', 'VOC_FORMS')

In [3]:
## Grid Search Params ##
MODELS = custom_models.get_models(input_size=(C, W, H))
MODELS = [
    MODELS['MLP'], 
    MODELS['SMALLER_MLP_2'], 
    MODELS['SMALLER_MLP_3'], 
    MODELS['SMALLER_MLP_3_3'], 
    MODELS['PERCEPTRON'], 
    MODELS['RESNET_34']
]
MODELS = map(lambda p: p.name, MODELS)
MODELS = list(MODELS)

DATASETS = [DT_DEST_RGB_RANDOM, DT_DEST_RGB_SINGLE_CLASS('AEROPLANE')]
OPTIMS = ["ADAM", "SGD"]
LOSS_FNS = ["L1LOSS"]

In [4]:
grid = model_helpers.new_grid_search(MODELS, OPTIMS, LOSS_FNS)
grid = list(grid)

print(f"Will train {len(LOSS_FNS) * len(MODELS) * len(DATASETS) * len(OPTIMS)} models")
print(f"{len(MODELS)} MODELS")
print(f"{len(DATASETS)} DATASETS")
print(f"{len(LOSS_FNS)} LOSS_FNS")
print(f"{len(OPTIMS)} OPTIMS")
print(f"Device: {DEVICE}")

Will train 24 models
6 MODELS
2 DATASETS
1 LOSS_FNS
2 OPTIMS
Device: cuda


In [5]:
curr_time = datetime.datetime.now()
CURR_TIME_STR = (f"{curr_time.year}-{curr_time.month}-{curr_time.day}_"
                 f"{curr_time.hour}-{curr_time.minute}-{curr_time.second}")
MAX_EPOCHS = 50
BS = 32

BASE_ARGS = {
    "C"        : C,
    "H"        : H,
    "W"        : W,
    "bs"       : BS,
    "epochs"   : MAX_EPOCHS,
    "device"   : DEVICE,
    "id"       : CURR_TIME_STR
}

print(f"Epochs: {MAX_EPOCHS}")
print(f"BS: {BS}")
print(f"Timestamp: {CURR_TIME_STR}")

Epochs: 50
BS: 32
Timestamp: 2019-12-23_17-2-35


In [7]:
def grid_search(dts, rows, sanity):
    if sanity: print("Performing sanity check...")
    else     : print("Training...")
    for dt in tqdm(dts):
        for row in tqdm(rows):
            command = (
                f'python3 {TRAIN_SINGLE_PATH} ' + 
                create_arg_str({
                    **BASE_ARGS,
                    "dataset": dt,
                    "model"  : row.model,
                    "optim"  : row.opt,
                    "loss_fn": row.loss,
                    "epochs" : MAX_EPOCHS,
                    "sanity" : sanity,
                }) + f' >> logs/out_{CURR_TIME_STR}.log')
            print(command)
            status = os.system(command)
            if status != 0: raise AssertionError(f'FAILED: {command}')
    if sanity: print("Sanity Check: All Passed!")
    else     : print("Done Training!")

grid_search(DATASETS, grid, sanity=True)
grid_search(DATASETS, grid, sanity=False)



  0%|          | 0/2 [00:00<?, ?it/s]


  0%|          | 0/12 [00:00<?, ?it/s]

Performing sanity check...
python3 ./train_single_script.py -model MLP -optim ADAM -loss_fn L1LOSS -dataset /home/victor/datasets/VOC_FORMS_RGB -bs 32 -epochs 50 -device cuda -C 3 -W 224 -H 224 -id 2019-12-23_17-2-35 --sanity >> logs/out_2019-12-23_17-2-35.log





  8%|▊         | 1/12 [00:04<00:45,  4.11s/it]

python3 ./train_single_script.py -model MLP -optim SGD -loss_fn L1LOSS -dataset /home/victor/datasets/VOC_FORMS_RGB -bs 32 -epochs 50 -device cuda -C 3 -W 224 -H 224 -id 2019-12-23_17-2-35 --sanity >> logs/out_2019-12-23_17-2-35.log





 17%|█▋        | 2/12 [00:08<00:41,  4.10s/it]

...

python3 ./train_single_script.py -model RESNET_34 -optim ADAM -loss_fn L1LOSS -dataset /home/victor/datasets/VOC_FORMS_RGB_AEROPLANE -bs 32 -epochs 50 -device cuda -C 3 -W 224 -H 224 -id 2019-12-23_17-2-35 --sanity >> logs/out_2019-12-23_17-2-35.log





 92%|█████████▏| 11/12 [00:39<00:03,  3.81s/it]

python3 ./train_single_script.py -model RESNET_34 -optim SGD -loss_fn L1LOSS -dataset /home/victor/datasets/VOC_FORMS_RGB_AEROPLANE -bs 32 -epochs 50 -device cuda -C 3 -W 224 -H 224 -id 2019-12-23_17-2-35 --sanity >> logs/out_2019-12-23_17-2-35.log





100%|██████████| 12/12 [00:44<00:00,  3.71s/it]


100%|██████████| 2/2 [01:29<00:00, 44.52s/it]

Sanity Check: All Passed!
